In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
%matplotlib inline

In [20]:
base_dir = "./data/"

In [21]:
def select_sample(data, n=1):
    np.random.shuffle(data)
    negatives = []
    positives = []
    straights = []
    for d in data:
        if d[1] > 0:
            positives.append(d)
        elif d[1] < 0:
            negatives.append(d)
        else:
            straights.append(d)
    return np.array(negatives[:n] + positives[:n] + straights[:n])

def get_model():
    ch, row, col = 3, 160, 320  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/255.0,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse", metrics=['mse'])
    return model
    
def data_gen(data):
    while 1:
        np.random.shuffle(data)
        for d in data:
            img = plt.imread(base_dir + d[0])
            #res = cv2.resize(img,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_AREA)
            res = np.expand_dims(img, axis=0)
            yield res, np.array([d[1]])
            
def normalize(X):
    ##keras code
    channel_index = 3
    row_index = 1
    col_index = 2
    mean = np.mean(X, axis=(0, row_index, col_index))
    broadcast_shape = [1, 1, 1]
    broadcast_shape[channel_index - 1] = X.shape[channel_index]
    mean = np.reshape(mean, broadcast_shape)
    print("mean: {}".format(mean))
    X -= mean
    
    std = np.std(X, axis=(0, row_index, col_index))
    broadcast_shape = [1, 1, 1]
    broadcast_shape[channel_index - 1] = X.shape[channel_index]
    std = np.reshape(std, broadcast_shape)
    print("std: {}".format(std))
    X /= (std + K.epsilon())
        
def predict_gen(data):
    while 1:
        for d in data:
            img = plt.imread(base_dir + d[0])
            #res = cv2.resize(img,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_AREA)
            res = np.expand_dims(img, axis=0)
            yield res
            
def read_data(array):
    array_size = array.shape[0]
    X_train = np.zeros((array_size, 160, 320, 3))
    y_train = np.zeros(array_size)
    print(X_train.shape)
    for i, x in enumerate(array):
        img = plt.imread(base_dir + x[0])
        X_train[i,:,:,:] = img
        y_train[i] = x[1]
    return X_train, y_train 

In [22]:
data = pd.read_csv("./data/driving_log.csv")[['center', 'steering']].values
sample = select_sample(data)
X_train, y_train = read_data(sample)

(3, 160, 320, 3)


In [23]:
normalize(X_train)

mean: [[[ 138.0413151   142.7535612   135.02099609]]]
std: [[[ 50.36909109  48.67878844  60.72792301]]]


In [5]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
datagen.fit(X_train)

In [11]:
??ImageDataGenerator

In [7]:
training_data = sample
epochs = 50
validation_data = sample


model = get_model()
model.fit_generator(datagen.flow(X_train, y_train, batch_size=3), samples_per_epoch=training_data.shape[0],
                    nb_epoch=epochs, validation_data=datagen.flow(X_train, y_train, batch_size=32),
                    nb_val_samples=validation_data.shape[0])

Epoch 1/50
3/3 [==============================] - 0s - loss: 0.0413 - mean_squared_error: 0.0413 - val_loss: 0.0317 - val_mean_squared_error: 0.0317
Epoch 2/50
3/3 [==============================] - 0s - loss: 0.0323 - mean_squared_error: 0.0323 - val_loss: 0.0174 - val_mean_squared_error: 0.0174
Epoch 3/50
3/3 [==============================] - 0s - loss: 0.0152 - mean_squared_error: 0.0152 - val_loss: 0.0220 - val_mean_squared_error: 0.0220
Epoch 4/50
3/3 [==============================] - 0s - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.0272 - val_mean_squared_error: 0.0272
Epoch 5/50
3/3 [==============================] - 0s - loss: 0.0306 - mean_squared_error: 0.0306 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 6/50
3/3 [==============================] - 0s - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0024 - val_mean_squared_error: 0.0024
Epoch 7/50
3/3 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024 - val_los

In [8]:
model.predict_generator(datagen.flow(X_train, y_train, batch_size=3, shuffle=False), training_data.shape[0])

array([[-0.00713443],
       [ 0.3561503 ],
       [ 0.02554228]], dtype=float32)

In [9]:
training_data

array([['IMG/center_2016_12_01_13_46_25_837.jpg', -0.05975719],
       ['IMG/center_2016_12_01_13_33_45_938.jpg', 0.3488158],
       ['IMG/center_2016_12_01_13_41_24_395.jpg', 0.0]], dtype=object)